In [3]:
# Imports
import mne
import numpy as np
import matplotlib.pyplot as plt
import nelpy as nel
import nelpy.plotting as npl
import scipy.io
from scipy import signal
from copy import *
import pandas as pd
from ghost.wave import ContinuousWaveletTransform
import ghost.wave as gwave
from seegpreprocessing import *

import warnings
warnings.filterwarnings('ignore')

In [4]:
%matplotlib qt

# Loading the Data

If grabbing signal from `data`, note that `data[channel]` will return a tuple, where the array at index `0` contains the values of the signal, and the array at index `1` contains the timestamps.

In [9]:
# Loading SEEG Data
path = '/home/ariel/Documents/CMU/AHN/May17_test.edf'
data = mne.io.read_raw_edf(path, preload=True)
raw_data = data.get_data()

# Loading corresponding .mat file
pat_csv = pd.read_csv('/home/ariel/Documents/CMU/AHN/p1_exp1.csv')

Extracting EDF parameters from /home/ariel/Documents/CMU/AHN/May17_test.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3125119  =      0.000 ...  3051.874 secs...


## Extracting trigger events and metadata

In [10]:
channel_mapping = {'gyrus rectus': ['O1','O2','O3','O4','O5','O6','O7','O8','O9','O10','O11','O12'], 
                   'insula': ['Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10','Y11','Y12','Y13', 'Y14'],
                   'aMCC': ['G1','G2','G3','G4','G5','G6','G7','G8','G9','G10','G11','G12'],
                   'STG': ['T1','T2','T3','T4'],
                   'temporal pole': ['I1','I2','I3','I4','I5','I6','I7','I8'],
                   'amygdala': ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10'],
                   'anterior hippocampus': ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10'],
                   'posterior hippocampus': ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10'],
                   'heschl gyrus': ['U1','U2','U3','U4','U5','U6','U7','U8'],
                   'fusiform gyrus': ['F1','F2','F3','F4','F5','F6','F7','F8'],
                   'triggers' : ['DC7', 'TRIG']}

In [11]:
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names
trig_idx = np.where(np.array(channels) == 'TRIG')[0][0]
trig_asa = nel.AnalogSignalArray(raw_data[trig_idx,:], fs=1024)

In [12]:
# Find event triggers (rising and falling edge)
trigger_events = mne.find_events(data, stim_channel="TRIG", output='step')
trigger_epochs = nel.EpochArray(trigger_events[:,0].reshape(90,2)/1024)

Trigger channel has a non-zero initial value of 255 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
180 events found
Event IDs: [0 1]


### Extract event type triggers

In [13]:
key_press_epochs = trigger_epochs[np.arange(2,90,6)]
display_epochs = trigger_epochs[np.arange(1,90,6)]

In [8]:
# Validating that the trigger events are visible
npl.plot(trig_asa)

# Plotting the key presses and expanding slightly so they color the full pulse
npl.plot(trig_asa[key_press_epochs.expand(0.001)], color='orange')

# Formatting the plot for readability
plt.legend(['trigger pulses', 'key presses'])
ax = plt.gca()
ax.set_xlim(left=trigger_epochs.expand(1).start, right=trigger_epochs.expand(1).stop)
ax.set_ylim(top=2, bottom=-1)
plt.title('Trigger Trace')

Text(0.5, 1.0, 'Trigger Trace')

## Constructing representations of SEEG signals

In [14]:
# Build AnalogSignalArrays to contain data of interest

# seeg_asa = nel.AnalogSignalArray(raw_data[:96,:], fs=1024)
amcc_asa = nel.AnalogSignalArray(raw_data[26:38,:], fs=1024)
ant_hippo_asa = nel.AnalogSignalArray(raw_data[60:70,:], fs=1024)
post_hippo_asa = nel.AnalogSignalArray(raw_data[70:80,:], fs=1024)

In [10]:
window = nel.EpochArray([trigger_epochs[0].start-0.5, trigger_epochs[1].stop+0.5])
with npl.FigureManager(show=True, figsize=(20,10)):
    ax = plt.gca()
    for idx, sig in enumerate(ant_hippo_asa.signals):
        npl.plot(sig[window], ax=ax, label=str(idx+1))
    npl.epochplot(trigger_epochs[0:2], fc='purple', ec='purple', alpha=0.2, ax=ax)
    ax.set_ylim(top=(ant_hippo_asa[window].data[:,:]).max())
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    
    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=5)
    plt.title('Anterior Hippocampus Trigger Event')

## Validating `.mat` data
We want to make sure we have access to the behavioral data and can visualize the information stored easily in Python

In [9]:
pat_csv

,color,word,char,display_time,wait_time,total_trial_time
0,blue,green,3,3.386251,0.479237,4.886824
1,red,blue,1,2.378658,0.602280,3.998565
2,red,green,1,1.115505,1.233669,3.369640
3,green,green,2,0.986795,0.922113,2.898775
4,green,red,2,1.090888,0.586531,2.684507
5,red,red,1,1.158711,0.903755,3.039829
6,blue,red,3,1.313497,0.831120,3.160708
7,blue,green,3,1.434117,0.942387,3.352748
8,red,blue,1,1.250112,1.321807,3.545485
9,red,green,1,1.981220,0.634936,3.582318


# Removing noise
Simple preprocessing of SEEG data, we are high pass filtering (default is at 0.5 Hz), and notching at harmonics of the line noise frequency (default is at 60 Hz).

In [15]:
def band_noise_filter(asa_in, lower=0.1, upper=360, noise_freq=60):
    '''
    Inputs:
        asa_in     - the AnalogSignalArray to be bandpass filtered and notched for noise removal.
        lower      - the lower cutoff frequency for the bandpass filter. Must be greater than 0 Hz.
        upper      - the upper cutoff frequency for the bandpass filter.
        noise_freq - the frequency to be removed
    
    Outputs:
        An AnalogSignalArray containing the bandpass filtered, notched signal with the same support
        as the input signal.
    '''

    # High pass filter to remove slow signal drift
    b,a = signal.butter(4, lower, btype='highpass', fs=asa_in.fs)
    hpass = scipy.signal.filtfilt(b, a, asa_in.data)
    denoising = deepcopy(hpass)

    # Notch filter to remove 60 Hz noise and its harmonics
    for num in range(1, int(asa_in.fs/(2*noise_freq))):
        harmonic = num*noise_freq
        b, a = signal.iirnotch(harmonic, 100, asa_in.fs)
        notch_inc = scipy.signal.filtfilt(b, a, denoising)
        denoising = deepcopy(notch_inc)

    # Create ASA with same support as asa_in, but change signal content
    no_stim_band_noiseless = deepcopy(asa_in)
    no_stim_band_noiseless.data = denoising
    return no_stim_band_noiseless

In [16]:
ant_hippo_filtered = band_noise_filter(ant_hippo_asa, lower=5, upper=512)
post_hippo_filtered = band_noise_filter(post_hippo_asa)
amcc_filtered = band_noise_filter(amcc_asa)

In [14]:
# Validating 60 Hz noise and harmonics were removed
with npl.FigureManager(show=True, figsize=(20,10)):
    npl.psdplot(ant_hippo_asa.signals[0])
    npl.psdplot(ant_hippo_filtered.signals[0])
    plt.title('PSD before and after filtering')

In [15]:
with npl.FigureManager(show=True, figsize=(20,10)):
    ax = plt.gca()
    for idx, sig in enumerate(ant_hippo_filtered.signals):
        npl.plot(sig[trigger_epochs[0].expand(.5)], ax=ax, label=str(idx+1))
    npl.epochplot(trigger_epochs[0], fc='purple', ec='purple', alpha=0.2, ax=ax)
    ax.set_ylim(top=(ant_hippo_filtered[trigger_epochs[0].expand(0.5)].data[:,:]).max())
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    
    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=5)
    plt.title('Anterior Hippocampus Trigger Event')
    plt.show()

# Keypress Average
We want to see how activity is stereotyped around the decision point -- here, we will average the trials at the point of key press (decision)

## Single channel

In [16]:
test_sig = ant_hippo_filtered.signals[1]
press_avg = np.zeros(1024)
ax = plt.gca()
for press in key_press_epochs:
    press_sig = test_sig[nel.EpochArray([press.start-0.5, press.start+0.5])].data.squeeze()
    plt.plot(np.linspace(-0.5,0.5,1024), press_sig, color='black', alpha=0.25)
    press_avg += press_sig
press_avg /= key_press_epochs.n_intervals
plt.plot(np.linspace(-0.5,0.5,1024), press_avg, color='red')
plt.axvline(0)
ax.set_xlim(left=-0.5, right=0.5)
plt.xlabel('Time from trigger onset')
plt.title('Anterior Hippocampus at Key Press')
plt.show()

## Multi channel (depth)

In [48]:
num_channels = len(ant_hippo_filtered.signals)
offset = 2
data_size = int((2*offset)*1024)
amcc_env = nel.utils.signal_envelope_1d(ant_hippo_filtered)
# env_channel = amcc_env.signals[idx]
for idx, channel in enumerate(ant_hippo_filtered.signals):
    splot = plt.subplot(num_channels, 1, idx+1)
    ax = plt.gca()
    press_avg = np.zeros(data_size)
    ax_twin = ax.twinx()
    env_channel = amcc_env.signals[idx]
    for press in key_press_epochs:
        press_sig = channel[nel.EpochArray([press.start-offset, press.start+offset])].data.squeeze()
        plt.plot(np.linspace(-offset,offset,data_size), press_sig, color='black', alpha=0.25)
        press_avg += env_channel[nel.EpochArray([press.start-offset, press.start+offset])].data.squeeze()
        # plt.setp(ax.get_xticklabels(), visible=False)
    press_avg /= key_press_epochs.n_intervals
    plt.plot(np.linspace(-offset,offset,data_size), press_avg, color='red')
    # plt.tick_params('x', labelbottom=False)
    plt.axvline(0)
    ax.set_xlim(left=-offset, right=offset)
    if idx+1 == num_channels:
        plt.setp(ax.get_xticklabels(), visible=True)
        plt.xlabel('Key press offset (s)')
    else:
        plt.setp(ax.get_xticklabels(), visible=False)
plt.suptitle('Anterior hippocampus average response to keypress trigger')

Text(0.5, 0.98, 'Anterior hippocampus average response to keypress trigger')

In [60]:
num_epochs = display_epochs.n_intervals
offset = 1
data_size = int((2*offset)*1024)
channel = ant_hippo_filtered.signals[1]
for idx, epoch in enumerate(display_epochs):
    splot = plt.subplot(num_epochs, 1, idx+1)
    ax = plt.gca()
    press_sig = channel[nel.EpochArray([epoch.start-offset, epoch.start+offset])].data.squeeze()
    plt.plot(np.linspace(-offset,offset,data_size), press_sig, color='black', alpha=0.25)
    plt.axvline(0)
    ax.set_xlim(left=-offset, right=offset)
    if idx+1 < num_epochs:
        plt.setp(ax.get_xticklabels(), visible=False)
        
    else:
        plt.setp(ax.get_xticklabels(), visible=True)
        plt.xlabel('Key press offset (s)')

plt.suptitle('Anterior Hippocampus response to keypress trigger')

Text(0.5, 0.98, 'Anterior Hippocampus response to keypress trigger')

In [68]:
num_epochs = display_epochs.n_intervals
offset = 1
data_size = int((2*offset)*1024)
channel = ant_hippo_filtered.signals[1]
for idx, epoch in enumerate(display_epochs):
    splot = plt.subplot(num_epochs, 1, idx+1)
    ax = plt.gca()
    press_sig = channel[nel.EpochArray([epoch.start-offset, epoch.start+offset])]
    npl.psdplot(press_sig)
    plt.axvline(0)
    ax.set_xlim(left=-offset, right=offset)
    if idx+1 < num_epochs:
        plt.setp(ax.get_xticklabels(), visible=False)
        
    else:
        plt.setp(ax.get_xticklabels(), visible=True)
        plt.xlabel('Key press offset (s)')

plt.suptitle('Anterior Hippocampus response to keypress trigger')
# npl.psdplot(ant_hippo_filtered.signals[1][display_epochs[0]])

Text(0.5, 0.98, 'Anterior Hippocampus response to keypress trigger')

In [66]:
press_sig

array([ 5.56814339e-07,  4.02922931e-07,  4.76390825e-06, ...,
       -4.88327740e-05, -4.94125469e-05, -4.71798635e-05])

# Looking at frequency domain

In [24]:
cwt = ContinuousWaveletTransform()
cwt.transform(ant_hippo_filtered.signals[1])

In [35]:
cwt.power.shape

(142, 3125120)

In [43]:
key_press_epochs[0].

TypeError: 'NoneType' object is not callable

In [50]:
ep = key_press_epochs[4]
ax = plt.gca()
ax.set_ylim(top=360,bottom=5)
cwt.plot(logscale=False,
         standardize=True,
         cmap=plt.cm.Spectral_r,
         levels=300,
         vmin=0,
         vmax=10,
         ax=ax,
         time_limits=nel.EpochArray([ep.start-2, ep.start+2]))

# Sharp wave ripples?

In [58]:
def ripple_band_filter(asa_in, lower=80, upper=140):
    '''
    Inputs:
        asa_in     - the AnalogSignalArray to be bandpass filtered and notched for noise removal.
        lower      - the lower cutoff frequency for the bandpass filter. Must be greater than 0 Hz.
        upper      - the upper cutoff frequency for the bandpass filter.
        noise_freq - the frequency to be removed
    
    Outputs:
        An AnalogSignalArray containing the bandpass filtered, notched signal with the same support
        as the input signal.
    '''

    # High pass filter to remove slow signal drift
    b,a = signal.butter(4, [lower, upper], btype='bandpass', fs=asa_in.fs)
    bpass = scipy.signal.filtfilt(b, a, asa_in.data)

    # Create ASA with same support as asa_in, but change signal content
    rband = deepcopy(asa_in)
    rband.data = bpass
    return rband

In [52]:
ant_hip_rband = ripple_band_filter(ant_hippo_filtered)
post_hip_rband = ripple_band_filter(post_hippo_filtered)

In [53]:
ant_hippo_rband_env = nel.utils.signal_envelope_1d(ant_hip_rband)
post_hippo_rband_env = nel.utils.signal_envelope_1d(post_hip_rband)

In [54]:
num_channels = len(ant_hippo_rband_env.signals)
offset = 2
data_size = int((2*offset)*1024)
for idx, channel in enumerate(ant_hippo_rband_env.signals):
    splot = plt.subplot(num_channels, 1, idx+1)
    ax = plt.gca()
    press_avg = np.zeros(data_size)
    ax_twin = ax.twinx()
    for press in key_press_epochs:
        press_sig = channel[nel.EpochArray([press.start-offset, press.start+offset])].data.squeeze()
        plt.plot(np.linspace(-offset,offset,data_size), press_sig, color='black', alpha=0.25)
        press_avg += press_sig
        # plt.setp(ax.get_xticklabels(), visible=False)
    press_avg /= key_press_epochs.n_intervals
    plt.plot(np.linspace(-offset,offset,data_size), press_avg, color='red')
    # plt.tick_params('x', labelbottom=False)
    plt.axvline(0)
    ax.set_xlim(left=-offset, right=offset)
    if idx+1 == num_channels:
        plt.setp(ax.get_xticklabels(), visible=True)
        plt.xlabel('Key press offset (s)')
    else:
        plt.setp(ax.get_xticklabels(), visible=False)
plt.suptitle('Anterior Hippocampus ripple band average response to keypress trigger')

Text(0.5, 0.98, 'Anterior Hippocampus ripple band average response to keypress trigger')

In [56]:
cwt = ContinuousWaveletTransform()
cwt.transform(ant_hip_rband.signals[1])

In [59]:
ep = key_press_epochs[4]
ax = plt.gca()
ax.set_ylim(bottom=50,top=200)
cwt.plot(logscale=False,
         standardize=True,
         cmap=plt.cm.Spectral_r,
         levels=300,
         vmin=0,
         vmax=10,
         ax=ax,
         time_limits=nel.EpochArray([ep.start-2, ep.start+2]))

## Wavelets

In [14]:
cwt = ContinuousWaveletTransform()
cwt.transform(amcc_filtered.signals[7])

In [18]:
cwt.power.shape

(142, 3125120)

In [24]:
ep = key_press_epochs[3]
ax = plt.gca()
ax.set_ylim(top=300)
cwt.plot(logscale=False,
         standardize=True,
         cmap=plt.cm.Spectral_r,
         levels=300,
         vmin=0,
         vmax=10,
         ax=ax,
         time_limits=nel.EpochArray([ep.start-0.5, ep.start+0.5]))

In [ ]:
with npl.FigureManager(show=True, figsize=(45,45)):
    for idx, press in enumerate(key_press_epochs):
        # press_sig = test_sig[nel.EpochArray([press.start-0.5, press.start+0.5])].data.squeeze()
        # plt.plot(np.linspace(-0.5,0.5,1024), press_sig, color='black', alpha=0.25)
        splot = plt.subplot(3,5,idx+1)
        cwt.plot(logscale=False,
         standardize=True,
         cmap=plt.cm.Spectral_r,
         levels=300,
         vmin=0,
         vmax=10,
         time_limits=nel.EpochArray([press.start-0.5, press.start+0.5]))
    # plt.savefig(r"C:\Users\Windows\Documents\CMU\CMI\SCS04\plots\tri_med.png", transparent=True)

In [ ]:
amcc_filtered